## Preparation

In [1]:
%%capture
!conda install -c pykaldi pykaldi -y

In [2]:
!pip install kenlm

     |▊                               | 30 kB 1.8 MB/s eta 0:00:01

     |█▋                              | 71 kB 998 kB/s eta 0:00:02

     |██▎                             | 102 kB 1.2 MB/s eta 0:00:02

     |█████                           | 225 kB 1.2 MB/s eta 0:00:02

     |██████████▍                     | 471 kB 1.2 MB/s eta 0:00:01

...

 -

 \

 |

 /

 -

In [ ]:
%cd /tmp

In [ ]:
!git clone https://github.com/jimregan/fairseq/ --branch issue3581

In [ ]:
!git clone https://github.com/kpu/kenlm

In [ ]:
%%capture
!apt-get -y install libeigen3-dev liblzma-dev zlib1g-dev libbz2-dev

In [ ]:
%%capture
%cd kenlm
!mkdir build
%cd build
!cmake ..
!make -j 4
%cd /tmp

In [ ]:
import os
os.environ['PATH'] = f"{os.environ['PATH']}:/tmp/kenlm/build/bin/"
os.environ['FAIRSEQ_ROOT'] = '/tmp/fairseq'

In [ ]:
%cd /tmp/fairseq/

In [ ]:
%%capture
!python setup.py install

In [ ]:
%cd /tmp/fairseq/

In [ ]:
os.environ['HYDRA_FULL_ERROR'] = '1'

## GAN

In [ ]:
!ls /kaggle/input/wav2vec-u-cv-swedish-audio/

In [ ]:
%%writefile rungan.sh
PREFIX=w2v_unsup_gan_xp
#TASK_DATA=/path/to/features/unfiltered/precompute_unfiltered_pca512_cls128_mean_pooled
TASK_DATA=/kaggle/input/wav2vec-u-cv-swedish-audio/precompute_pca512_cls128_mean_pooled/
#TEXT_DATA=/path/to/data  # path to fairseq-preprocessed GAN data
TEXT_DATA=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/
#KENLM_PATH=/path/to/data/kenlm.phn.o4.bin  # KenLM 4-gram phoneme language model (LM data = GAN data here)
KENLM_PATH=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/lm.phones.filtered.04.bin

PREFIX=$PREFIX fairseq-hydra-train \
	-m --config-dir fairseq/config/model/wav2vecu/gan \
	--config-name w2vu \
	task.data=${TASK_DATA} \
	task.text_data=${TEXT_DATA} \
	task.kenlm_path=${KENLM_PATH} \
	checkpoint.no_epoch_checkpoints=false \
	checkpoint.save_dir=/kaggle/working \
	'common.seed=range(0,5)'

In [ ]:
!bash rungan.sh